In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

In [2]:
output = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/01_Text/04_Abbildungen/Kapitel_4/google_transit/trips.shp"

# Read-In data

In [3]:
stops = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/01_Text/04_Abbildungen/Kapitel_4/google_transit/stops.txt"
shapes = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/01_Text/04_Abbildungen/Kapitel_4/google_transit/shapes.txt"
trips = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/01_Text/04_Abbildungen/Kapitel_4/google_transit/trips.txt"
routes = "C:/Users/david/OneDrive/02_Uni/02_Master/05_Masterarbeit/01_Text/04_Abbildungen/Kapitel_4/google_transit/routes.txt"

In [4]:
df_stops = pd.read_csv(stops)
df_shapes = pd.read_csv(shapes)
df_trips = pd.read_csv(trips)
df_routes = pd.read_csv(routes)

In [5]:
df_stops.head(3)

,stop_id,stop_name,stop_lat,stop_lon
0,de:08111:100:1:1,Stammheim,48.850046,9.156174
1,de:08111:100:2:3,Stammheim,48.849443,9.156022
2,de:08111:100:2:4,Stammheim,48.850217,9.156093


In [6]:
df_shapes.head(3)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,10-12-j21-1.1.H,48.806804,9.525427,1,0.00
1,10-12-j21-1.1.H,48.806196,9.523318,2,168.63
2,10-12-j21-1.1.H,48.805813,9.522288,3,255.25


In [7]:
df_trips.head(3)

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id,block_id
0,10-12-j21-1,T0#10,100.T0.10-12-j21-1.13.R,10-12-j21-1.13.R,Schorndorf,1,NaN
1,10-12-j21-1,T0#10,11.T0.10-12-j21-1.6.H,10-12-j21-1.6.H,Esslingen (N),0,NaN
2,10-12-j21-1,T0#10,13.T0.10-12-j21-1.6.H,10-12-j21-1.6.H,Esslingen (N),0,NaN


In [8]:
df_routes.head(3)

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,10-12-j21-1,NaN,S12,Schorndorf - Bad Cannstatt - Esslingen,109,83b23b,FFFFFF
1,10-15-j21-1,NaN,S15,Bietigheim - Feuerbach - Vaihingen - Herrenberg,109,83b23b,FFFFFF
2,10-15-j21-2,NaN,S15,Bietigheim - Feuerbach - Vaihingen - Herrenberg,109,83b23b,FFFFFF


In [9]:
df_routes[df_routes['route_short_name'] == 'S1']

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
5,10-1-j21-1,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF
6,10-1-j21-13,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF
7,10-1-j21-14,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF
8,10-1-j21-15,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF
9,10-1-j21-16,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF
10,10-1-j21-17,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF
11,10-1-j21-18,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF
12,10-1-j21-19,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF
13,10-1-j21-22,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF
14,10-1-j21-23,NaN,S1,Kirchheim (T) - Plochingen - Stuttgart - Herre...,109,83b23b,FFFFFF


# Create line shapes

In [10]:
gdf_shapes = gpd.GeoDataFrame(
    df_shapes, geometry=gpd.points_from_xy(df_shapes.shape_pt_lon, df_shapes.shape_pt_lat))
gdf_shapes = gdf_shapes.set_crs(epsg=4326)

gdf_shapes.sort_values(by=['shape_id', 'shape_pt_sequence'], inplace=True)

df_lines = list()
uniques = gdf_shapes['shape_id'].unique()
total = len(uniques)
cnt = 0

for shape_id in uniques:
    df = gdf_shapes[gdf_shapes['shape_id'] == shape_id]
    lst = list()
    
    for index, row in df.iterrows():
        lst.append(row['geometry'])
        
    df_lines.append({'shape_id': shape_id, 'geometry': LineString(lst)})
    
    if cnt%100 == 0:
        print(str(cnt) + "/" + str(total))
    
    cnt = cnt + 1
    
df_lines = pd.DataFrame(df_lines)
gdf_lines = gpd.GeoDataFrame(
    df_lines.drop(columns=['geometry']), geometry=df_lines['geometry'])
gdf_lines = gdf_lines.set_crs(epsg=4326)

0/8404
100/8404
200/8404
300/8404
400/8404
500/8404
600/8404
700/8404
800/8404
900/8404
1000/8404
1100/8404
1200/8404
1300/8404
1400/8404
1500/8404
1600/8404
1700/8404
1800/8404
1900/8404
2000/8404
2100/8404
2200/8404
2300/8404
2400/8404
2500/8404
2600/8404
2700/8404
2800/8404
2900/8404
3000/8404
3100/8404
3200/8404
3300/8404
3400/8404
3500/8404
3600/8404
3700/8404
3800/8404
3900/8404
4000/8404
4100/8404
4200/8404
4300/8404
4400/8404
4500/8404
4600/8404
4700/8404
4800/8404
4900/8404
5000/8404
5100/8404
5200/8404
5300/8404
5400/8404
5500/8404
5600/8404
5700/8404
5800/8404
5900/8404
6000/8404
6100/8404
6200/8404
6300/8404
6400/8404
6500/8404
6600/8404
6700/8404
6800/8404
6900/8404
7000/8404
7100/8404
7200/8404
7300/8404
7400/8404
7500/8404
7600/8404
7700/8404
7800/8404
7900/8404
8000/8404
8100/8404
8200/8404
8300/8404
8400/8404


In [11]:
gdf_lines.head(3)

,shape_id,geometry
0,10-1-j21-1.1.H,"LINESTRING (9.41079 48.71274, 9.40896 48.71460..."
1,10-1-j21-1.10.R,"LINESTRING (9.15655 48.77009, 9.15798 48.77046..."
2,10-1-j21-1.11.R,"LINESTRING (9.00456 48.68792, 9.00551 48.68845..."


# Merge

In [12]:
gdf_trips = df_trips.merge(gdf_lines, how ='left')
gdf_trips = gdf_trips.merge(df_routes, how = 'left')

In [13]:
gdf_trips = gpd.GeoDataFrame(
    gdf_trips.drop(columns=['geometry']), geometry=gdf_trips['geometry'])

In [14]:
gdf_trips = gdf_trips[['route_short_name', 'route_type', 'route_color', 'geometry']]
gdf_trips = gdf_trips[gdf_trips['geometry'] != None]

In [15]:
gdf_trips.head(3)

,route_short_name,route_type,route_color,geometry
0,S12,109,83b23b,"LINESTRING (9.30054 48.73843, 9.29817 48.73914..."
1,S12,109,83b23b,"LINESTRING (9.52547 48.80674, 9.52303 48.80603..."
2,S12,109,83b23b,"LINESTRING (9.52547 48.80674, 9.52303 48.80603..."


In [16]:
gdf_trips.to_file(output)